In [1]:
import gseapy as gp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import warnings
import plotly.graph_objects as go
import plotly.io as pio
from palettable.tableau import Tableau_20
warnings.filterwarnings('ignore')
sns.set_context("paper")
sns.set_theme(style="whitegrid", font_scale=0.8)
dpi = 600

In [2]:
def plot_KEGG(enr=None, top_nums=15, show=False, file_name=None):
    """
    绘制KEGG通路富集分析结果的条形图和气泡图
    :param enr: enrichr对象，包含富集分析结果
    :param top_nums: 要显示的通路数量
    :param file_name: 保存图表的文件名，如果为None则不保存
    :return:
    """
    results_df = enr.results.sort_values(by='Adjusted P-value').head(top_nums)

    # 美化通路名称
    def clean_term_names(term):
        return term.replace("KEGG_", "").replace("_", " ").capitalize()
    results_df['Term_clean'] = results_df['Term'].apply(clean_term_names)

    results_df['-log10(AdjP)'] = -np.log10(results_df['Adjusted P-value'])

    # 解析 'Overlap' 列
    results_df['Genes_in_set'] = results_df['Overlap'].apply(lambda x: int(x.split('/')[0]))

    """ --- 绘制条形图 --- """
    plt.figure(figsize=(7, 8))

    ax = sns.barplot(
        data=results_df.sort_values(by='-log10(AdjP)', ascending=False), # 使用新列名排序
        x='-log10(AdjP)', # 使用新列名作为x轴数据
        y='Term_clean',
        palette='viridis'
    )

    # --- 图表美化 ---
    # ax.set_title('KEGG Pathway Enrichment Analysis', fontsize=16, fontweight='bold')
    ax.set_xlabel(r'$-\log_{10}(\mathrm{Adjusted\ P-value})$', fontsize=12)
    ax.set_ylabel('')
    ax.tick_params(axis='y', labelsize=11)
    sns.despine()
    plt.tight_layout()

    if file_name:
        plt.savefig(f"{file_name}_bar.png", dpi=dpi, bbox_inches='tight')
    if show:
        plt.show()
    plt.close()

    """ --- 绘制气泡图 --- """
    fig = plt.figure(figsize=(6, 7))
    gs = gridspec.GridSpec(1, 2, width_ratios=[8, 2], wspace=0.1)
    ax = plt.subplot(gs[0])

    # 使用新列名作为hue
    sns.scatterplot(
        data=results_df,
        x='Combined Score',
        y='Term_clean',
        size='Genes_in_set',
        hue='-log10(AdjP)', # 使用新列名作为颜色映射
        palette='viridis_r',
        sizes=(50, 500),
        edgecolor='black',
        linewidth=0.5,
        ax=ax,
        legend=False
    )

    # 主图美化
    # ax.set_title('KEGG Pathway Enrichment Analysis', fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Combined Score', fontsize=12)
    ax.set_ylabel('')
    ax.tick_params(axis='y', labelsize=11)
    ax.grid(True, axis='x', linestyle='--', alpha=0.6)
    ax.margins(y=0.05)
    sns.despine(ax=ax)


    # --- 在画布的特定位置创建对齐的图例 ---
    x_pos = 0.82

    # 大小图例 (Gene Count)
    min_size = results_df['Genes_in_set'].min()
    max_size = results_df['Genes_in_set'].max()
    legend_sizes = [min_size, int((min_size + max_size) / 2), max_size]
    legend_handles = [plt.scatter([], [], s=s*20, color='gray', edgecolor='black', linewidth=0.5) for s in legend_sizes]
    size_legend = fig.legend(handles=legend_handles,
                             labels=[str(s) for s in legend_sizes],
                             bbox_to_anchor=(x_pos, 0.88),
                             loc='upper left',
                             title='Gene Count',
                             frameon=False,
                             fontsize=11,
                             title_fontsize=12,
                             labelspacing=1.2)

    # 颜色条图例
    cax = fig.add_axes([x_pos, 0.25, 0.03, 0.3])
    # 使用新列名设置颜色条范围
    norm = plt.Normalize(results_df['-log10(AdjP)'].min(), results_df['-log10(AdjP)'].max())
    sm = plt.cm.ScalarMappable(cmap="viridis_r", norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    cbar.ax.tick_params(labelsize=10)
    cbar.set_label(r'$\log_{10}\left(\frac{1}{\mathrm{FDR}}\right)$', rotation=270, labelpad=20, fontsize=12)

    if file_name:
        plt.savefig(f"{file_name}_dot.png", dpi=dpi, bbox_inches='tight')
    if show:
        plt.show()
    plt.close()

def plot_enrichment_sankey_dot(enr, top_nums=15, file_name=None, title='', show=False):
    """
    通过共享单Y轴和手动绘制，创建精确对齐、布局合理的出版级组合图。
    - 修正气泡图边框，使其精确包围气泡图内容区域。

    :param enr: gseapy 的 enrichr 对象。
    :param top_nums: 要显示的通路数量。
    :param file_name: 保存图表的文件名（无需扩展名）。
    :param title: 图表标题。
    """
    # 1. --- 数据准备 ---
    results_df = enr.results.sort_values(by='Adjusted P-value').head(top_nums).reset_index(drop=True)
    results_df['Term'] = results_df['Term'].str.replace("KEGG_", "").str.replace("_", " ").str.capitalize()
    results_df['Genes_list'] = results_df['Genes'].str.split(';')
    results_df['-log10(AdjP)'] = -np.log10(results_df['Adjusted P-value'])
    overlap_split = results_df['Overlap'].str.split('/', expand=True).astype(int)
    results_df['Gene Count'] = overlap_split[0]
    results_df['Gene Ratio'] = overlap_split[0] / overlap_split[1]

    sankey_df = results_df[['Term', 'Genes_list', 'Adjusted P-value']].explode('Genes_list').rename(columns={'Genes_list': 'Gene'})

    sankey_df['min_p_for_gene'] = sankey_df.groupby('Gene')['Adjusted P-value'].transform('min')
    sankey_df = sankey_df.sort_values(by=['min_p_for_gene', 'Gene'], ascending=[True, True])
    unique_genes = sankey_df['Gene'].unique().tolist()
    unique_terms = results_df['Term'].tolist()

    # 创建统一的数值Y轴坐标系
    num_genes = len(unique_genes)
    gene_y_map = {gene: num_genes - 1 - i for i, gene in enumerate(unique_genes)}

    pathway_cluster_ratio = 0.8  # 调整通路块的高度比例，更小使通路块更紧凑
    pathway_block_height = (num_genes - 1) * pathway_cluster_ratio if num_genes > 1 else 0
    term_y_positions = np.linspace(0, pathway_block_height, len(unique_terms))
    term_y_map = {term: term_y_positions[len(unique_terms) - 1 - i] for i, term in enumerate(unique_terms)}

    # 准备颜色
    term_colors = {term: color for term, color in zip(unique_terms, Tableau_20.hex_colors)}
    most_significant_term = sankey_df.loc[sankey_df.groupby('Gene')['Adjusted P-value'].idxmin()]
    gene_colors = {row['Gene']: term_colors.get(row['Term'], 'grey') for _, row in most_significant_term.iterrows()}

    # 2. --- 创建一个空的Figure ---
    fig = go.Figure()

    # 3. --- 手动绘制桑基图元素 ---
    layout_shapes = []
    # a. 绘制连接曲线
    for _, row in sankey_df.iterrows():
        y0, y1 = gene_y_map[row['Gene']], term_y_map[row['Term']]
        path = f'M 0,{y0} C 0.5,{y0} 0.5,{y1} 1,{y1}'
        layout_shapes.append(dict(type="path", path=path, line_color="lightgrey", line_width=1, layer="below", xref="x", yref="y"))

    # b. 绘制基因和通路的色块和标签
    for gene, y in gene_y_map.items():
        layout_shapes.append(dict(
            type="rect", xref="x", yref="y",
            x0=-0.09, y0=y-0.2,
            x1=-0.05, y1=y+0.2,
            fillcolor=gene_colors.get(gene, 'grey'),
            line_width=0
        ))
        fig.add_annotation(x=-0.11, y=y, text=gene, showarrow=False, xanchor="right", xref="x", yref="y")

    min_gene_count, max_gene_count = results_df['Gene Count'].min(), results_df['Gene Count'].max()
    min_height, max_height = 1.0, 3.0  # 色块的最小和最大高度

    for _, row in results_df.iterrows():
        term, y, gene_count = row['Term'], term_y_map[row['Term']], row['Gene Count']
        if max_gene_count > min_gene_count: norm_count = (gene_count - min_gene_count) / (max_gene_count - min_gene_count)
        else: norm_count = 0.5
        height = min_height + norm_count * (max_height - min_height)

        # 宽度：由 x0=1.01 和 x1=1.05 决定。色块的宽度就是 x1 - x0（即 0.04）。可以调整这两个值来改变宽度，例如 x0=1.01, x1=1.06 会让色块变得更宽。
        layout_shapes.append(dict(type="rect", xref="x", yref="y", x0=1.01, y0=y-(height/2), x1=1.05, y1=y+(height/2), fillcolor=term_colors.get(term, 'grey'), line_width=0))
        fig.add_annotation(x=1, y=y, text=term, showarrow=False, xanchor="right", xref="x", yref="y", xshift=-5, font=dict(size=20, color="black"))

    # 动态计算图例的Y轴位置
    if num_genes > 1:
        colorbar_y_pos = (max(term_y_map.values()) / (num_genes - 1)) + 0.05 if term_y_map else 0.05
    else:
        colorbar_y_pos = 0.9

    # 4. --- 添加气泡图轨迹 (Trace) ---
    fig.add_trace(go.Scatter(
        x=results_df['Gene Ratio'], y=results_df['Term'].map(term_y_map),
        mode='markers',
        marker=dict(
            size=25, color=results_df['-log10(AdjP)'], colorscale='YlOrRd',  # size调整气泡大小
            showscale=True,
            colorbar=dict(
                title='log<sub>10</sub>(1/FDR)', title_font=dict(size=20), orientation='h', x=0.825, xanchor='center',
                y=colorbar_y_pos, yanchor='bottom', len=0.25, title_side='top'
            )
        ),
        hoverinfo='x', xaxis='x2', yaxis='y'
    ))

    # 获取通路区域的实际Y轴范围
    y_min_pathway = min(term_y_map.values()) if term_y_map else 0
    y_max_pathway = max(term_y_map.values()) if term_y_map else 0

    # 添加一个白色矩形遮盖气泡图上方多余的网格线
    layout_shapes.append(
        dict(type="rect", xref="paper", yref="y", x0=0.64, y0=y_max_pathway + 1, # 从通路最高点上方开始
             x1=1.0, y1=num_genes, fillcolor="white", line_width=0, layer="above")
    )

    # # --- 核心修正：为气泡图区域添加精确对齐的边框 ---
    layout_shapes.append(
        dict(
            type="rect",
            xref="paper", yref="y",
            x0=0.64, # 气泡图区域的左侧paper坐标
            x1=0.98, # 气泡图区域的右侧paper坐标
            y0=y_min_pathway - 1, # <--- 修正：使用通路区域的最低点减去一点偏移
            y1=y_max_pathway + 1, # <--- 修正：使用通路区域的最高点加上一点偏移
            line=dict(color="black", width=1),
            fillcolor="rgba(0,0,0,0)",
            layer="below"
        )
    )

    # 5. --- 配置多X轴，单Y轴布局 ---
    fig.update_layout(
        title_text=title,
        height=max(600, num_genes * 16), width=1200,  # 根据基因数量动态调整高度
        template='plotly_white', font=dict(size=12),
        showlegend=False, shapes=layout_shapes,

        xaxis=dict(domain=[0, 0.63], visible=False, range=[-0.4, 1.1]),
        xaxis2=dict(domain=[0.64, 0.98], title=dict(text="Gene ratio", font=dict(size=20)), anchor='y'),
        # Y轴的range应该覆盖所有基因和通路的范围，包括为了视觉效果添加的少量上下边距
        yaxis=dict(domain=[0, 1], visible=False, range=[-1, num_genes])
    )

    # 添加哑元点以确保Y轴范围正确
    fig.add_trace(go.Scatter(x=[None], y=[num_genes-1, 0], mode='markers', marker_size=0, xaxis='x', yaxis='y'))

    # 6. --- 保存图表 ---
    if file_name:
        height_px = max(600, num_genes * 18)
        width_px = 1200
        # 定义HTML, SVG, 和 PNG 的文件名
        html_path = f"{file_name}_sankey+dot.html"
        svg_path = f"{file_name}_sankey+dot.svg"
        png_path = f"{file_name}_sankey+dot.png"
        pdf_path = f"{file_name}_sankey+dot.pdf"

        # 保存交互式HTML
        fig.write_html(html_path)
        print(f"Interactive HTML saved to {html_path}")
        try:
            # --- 新增：直接保存为PDF ---
            pio.write_image(fig, pdf_path, width=width_px, height=height_px)
            print(f"Vector PDF image saved to {pdf_path}")

            # 保存SVG
            pio.write_image(fig, svg_path, width=width_px, height=height_px)
            print(f"Vector SVG image saved to {svg_path}")

            # 保存PNG
            pio.write_image(fig, png_path, width=width_px, height=height_px, scale=3)
            print(f"High-resolution PNG image saved to {png_path}")

        except Exception as e:
             print(f"Could not save static image. You might need to install kaleido. Error: {e}")

    if show:
        fig.show()
    plt.close()

In [3]:
ppi_genes = set(pd.read_csv('../data/knowledge/PPI_data_min700.txt', sep='\t')['protein1'].tolist())
data_name = ['monaco_pbmc', 'sdy67', 'microarray', 'GSE107572', 'GSE120502', 'monaco2', 'sdy67_250', 'brain_human']
for name in data_name:
    print(f'Processing {name}...')
    features_list = []
    with open(f'../results/plot/SHAP/{name}/top200_knowledge_features.txt', 'r') as f:
        for line in f:
            features_list.append(line.strip())
    gene_list = list(set(features_list) & ppi_genes)
    enr = gp.enrichr(gene_list=gene_list,
                     gene_sets='../data/knowledge/c2.cp.kegg_legacy.v2024.1.Hs.symbols.gmt',
                     organism='Human',
                     cutoff=0.5
                     )

    # plot_KEGG(enr, top_nums=15, file_name=f'../results/plot/SHAP/{name}/KEGG_enrichment')

    plot_enrichment_sankey_dot(
        enr,
        top_nums=15,
        file_name=f'../results/plot/SHAP/{name}/KEGG_enrichment',
        # title=f'KEGG Pathway Enrichment for {name}'
    )


Processing monaco_pbmc...
Interactive HTML saved to ../results/plot/SHAP/monaco_pbmc/KEGG_enrichment_sankey+dot.html
Vector PDF image saved to ../results/plot/SHAP/monaco_pbmc/KEGG_enrichment_sankey+dot.pdf
Vector SVG image saved to ../results/plot/SHAP/monaco_pbmc/KEGG_enrichment_sankey+dot.svg
High-resolution PNG image saved to ../results/plot/SHAP/monaco_pbmc/KEGG_enrichment_sankey+dot.png
Processing sdy67...
Interactive HTML saved to ../results/plot/SHAP/sdy67/KEGG_enrichment_sankey+dot.html
Vector PDF image saved to ../results/plot/SHAP/sdy67/KEGG_enrichment_sankey+dot.pdf
Vector SVG image saved to ../results/plot/SHAP/sdy67/KEGG_enrichment_sankey+dot.svg
High-resolution PNG image saved to ../results/plot/SHAP/sdy67/KEGG_enrichment_sankey+dot.png
Processing microarray...
Interactive HTML saved to ../results/plot/SHAP/microarray/KEGG_enrichment_sankey+dot.html
Vector PDF image saved to ../results/plot/SHAP/microarray/KEGG_enrichment_sankey+dot.pdf
Vector SVG image saved to ../resul

In [4]:
ppi_genes = set(pd.read_csv('../data/knowledge/PPI_data_min700.txt', sep='\t')['protein1'].tolist())
data_name = ['CRC-sEV']
for name in data_name:
    for tissue in ['normal', 'cancer']:
        features_list = []
        with open(f'../results/plot/SHAP/{name}/{tissue}/top200_knowledge_features.txt', 'r') as f:
            for line in f:
                features_list.append(line.strip())
        gene_list = list(set(features_list) & ppi_genes)
        enr = gp.enrichr(gene_list=gene_list,
                         gene_sets='../data/knowledge/c2.cp.kegg_legacy.v2024.1.Hs.symbols.gmt',
                         organism='Human',
                         cutoff=0.5
                         )

        plot_KEGG(enr, top_nums=15, file_name=f'../results/plot/SHAP/{name}/{tissue}/KEGG_enrichment')

        plot_enrichment_sankey_dot(
            enr,
            top_nums=15,
            file_name=f'../results/plot/SHAP/{name}/{tissue}/KEGG_enrichment',
            # title=f'KEGG Pathway Enrichment for {name}'
    )


Interactive HTML saved to ../results/plot/SHAP/CRC-sEV/normal/KEGG_enrichment_sankey+dot.html
Vector PDF image saved to ../results/plot/SHAP/CRC-sEV/normal/KEGG_enrichment_sankey+dot.pdf
Vector SVG image saved to ../results/plot/SHAP/CRC-sEV/normal/KEGG_enrichment_sankey+dot.svg
High-resolution PNG image saved to ../results/plot/SHAP/CRC-sEV/normal/KEGG_enrichment_sankey+dot.png
Interactive HTML saved to ../results/plot/SHAP/CRC-sEV/cancer/KEGG_enrichment_sankey+dot.html
Vector PDF image saved to ../results/plot/SHAP/CRC-sEV/cancer/KEGG_enrichment_sankey+dot.pdf
Vector SVG image saved to ../results/plot/SHAP/CRC-sEV/cancer/KEGG_enrichment_sankey+dot.svg
High-resolution PNG image saved to ../results/plot/SHAP/CRC-sEV/cancer/KEGG_enrichment_sankey+dot.png
